## Driver Drowsiness detection with OpenCV


### The drowsiness detector algorithm

##### The general flow of drowsiness detection algorithm is fairly straightforward.

##### First, we’ll setup a camera that monitors a stream for faces:
1. Look for faces in the input video stream.
2. Apply facial landmark detection to extract the eye regions from the face.
3. Compute the eye aspect ratio to determine if the eyes are closed.
4. Sound an alarm if the eyes have been closed for a sufficiently long enough time.

Required Packages: 
1. <b>'OpenCV'</b>
2. The <b>'SciPy'</b> package is used to compute the Euclidean distance between facial landmarks points in the eye aspect ratio calculation.<
3. The <b>'Thread'</b> class is used to play alarm in a separate thread from the main thread to ensure our script doesn’t pause execution while the alarm sounds.
4. The <b>'playsound'</b> library is used in order to play WAV/MP3 alarm sound, which is a pure Python, cross-platform implementation for playing simple sounds.
5. The <b>'dlib'</b> library is used to detect and localize facial landmarks.

In [1]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import argparse
import imutils
import time
import dlib
import cv2

* The <b>sound_alarm()</b> function accepts a path to an audio file residing on disk and then plays the file.

In [2]:
def sound_alarm(path):
    # play an alarm sound
    playsound.playsound(path)

* The <b>eye_aspect_ratio()</b> function compute the ratio of distances between the vertical eye landmarks and the distances between the horizontal eye landmarks.
* The return value of the eye aspect ratio will be approximately constant when the eye is open. 
* The value will then rapid decrease towards zero during a blink.
* If the eye is closed, the eye aspect ratio will again remain approximately constant, but will be much smaller than the ratio when the eye is open.

![](../image/Drowsiness-Detection/blink_detection_plot.jpg)

* On the top-left we have an eye that is fully open with the eye facial landmarks plotted. 
* On the top-right we have an eye that is closed. 
* The bottom then plots the eye aspect ratio over time.

* As we can see, the eye aspect ratio is constant (indicating the eye is open), then rapidly drops to zero, then increases again, indicating a blink has taken place.

* In our drowsiness detector case, we’ll be monitoring the eye aspect ratio to see if the value falls but does not increase again, thus implying that the person has closed their eyes.

In [3]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear

In [4]:
# shape_predictor : This is the path to dlib’s pre-trained facial landmark detector. 
shape_predictor = "../models/Drowsiness Detection/shape_predictor_68_face_landmarks.dat"
#alarm : The path to an input audio file to be used as an alarm
alarm = "../alarm/alarm.wav"

* If the eye aspect ratio falls below 'EYE_AR_THRESH' value, we’ll start counting the number of frames the person has closed their eyes for. (An EYE_AR_THRESH  of 0.3  works well in a variety of situations)
* If the number of frames the person has closed their eyes in exceeds EYE_AR_CONSEC_FRAMES, we’ll sound an alarm.
* We have set the EYE_AR_CONSEC_FRAMES  to be 48, meaning that if a person has closed their eyes for 48 consecutive frames, we’ll play the alarm sound.
* You can make the drowsiness detector more sensitive by decreasing the EYE_AR_CONSEC_FRAMES — similarly, you can make the drowsiness detector less sensitive by increasing it.
* COUNTER, the total number of consecutive frames where the eye aspect ratio is below EYE_AR_THRESH.
* If COUNTER exceeds EYE_AR_CONSEC_FRAMES, then we’ll update the boolean ALARM_ON.

In [5]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EYE_AR_THRESH = 0.3
#EYE_AR_CONSEC_FRAMES = 48
EYE_AR_CONSEC_FRAMES = 20
# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
COUNTER = 0
ALARM_ON = False

* The dlib library ships with a Histogram of Oriented Gradients-based face detector along with a facial landmark predictor<br> we instantiate both of these in the following block:

In [6]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor)

[INFO] loading facial landmark predictor...


* The facial landmarks produced by dlib are an indexable list, as shown below:
![](../image/Drowsiness-Detection/facial_landmarks_68markup-768x619.jpg)

* Therefore, to extract the eye regions from a set of facial landmarks, we simply need to know the correct array slice indexes:

In [7]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

* Using these indexes, we’ll easily be able to extract the eye regions via an array slice.
* We are now ready to start the core of our drowsiness detector:

 1. Instantiate our VideoStream.
 2. Pause for a second to allow the camera sensor to warm up.
 3. Start looping over frames in the video stream.
 4. Read the next frame, which we then preprocess by resizing it to have a width of 450 pixels and converting it to grayscale.
 5. Apply dlib’s face detector to find and locate the face(s) in the image.<br><br>
 
 * The next step is to apply facial landmark detection to localize each of the important regions of the face:
 
 1. Loop over each of the detected faces. We assume that there is only one face — the driver. But we left this for  loop in here just in case we want to apply the technique to videos with more than one face.

 2. For each of the detected faces, we apply dlib’s facial landmark detector and convert the result to a NumPy array.

 3. Using NumPy array slicing we can extract the (x, y)-coordinates of the left and right eye, respectively.

 4. Given the (x, y)-coordinates for both eyes, we then compute their eye aspect ratios.

 5. Soukupová and Čech recommend averaging both eye aspect ratios together to obtain a better estimation.

 6. We can then visualize each of the eye regions on our frame  by using the cv2.drawContours function below — this is often   helpful when we are trying to debug our script and want to ensure that the eyes are being correctly detected and localized:
 
* Finally, we are ready to check to see if the person in our video stream is starting to show symptoms of drowsiness:
 
 1. Make a check to see if the eye aspect ratio is below the “blink/closed” eye threshold, EYE_AR_THRESH.

 2. If it is, we increment COUNTER, the total number of consecutive frames where the person has had their eyes closed.

 3. If COUNTER exceeds EYE_AR_CONSEC_FRAMES, then we assume the person is starting to doze off.

 4. Another check is made, this time to see if the alarm is on — if it’s not, we turn it on.

 5. Handle playing the alarm sound, provided an --alarm  path was supplied when the script was executed. 
    We take special care to create a separate thread responsible for calling sound_alarm  to ensure that our main program isn’t     blocked until the sound finishes playing.

 6. Draw the text DROWSINESS ALERT!  on our frame  — again, this is often helpful for debugging, especially if you are not using     the playsound  library.

 7. Finally, handle the case where the eye aspect ratio is larger than EYE_AR_THRESH, indicating the eyes are open. 
    If the eyes are open, we reset COUNTER  and ensure the alarm is off.

* The final code block in our drowsiness detector handles displaying the output frame  to our screen.

In [8]:
# start the video stream thread
print("[INFO] starting video stream thread...")
vs = VideoStream(0).start()
time.sleep(1.0)

# loop over frames from the video stream
while True:
# grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale
    # channels)
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale frame
    rects = detector(gray, 0)
    
    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0
        
        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            # if the eyes were closed for a sufficient number of
            # then sound the alarm
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                # if the alarm is not on, turn it on
                if not ALARM_ON:
                    ALARM_ON = True
                    # check to see if an alarm file was supplied,
                    # and if so, start a thread to have the alarm
                    # sound played in the background
                    if alarm != "":
                        t = Thread(target=sound_alarm,
                            args=(alarm,))                                   
                        t.deamon = True
                        t.start()
                # draw an alarm on the frame
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        # otherwise, the eye aspect ratio is not below the blink
        # threshold, so reset the counter and alarm
        else:
            COUNTER = 0
            ALARM_ON = False
            
        # draw the computed eye aspect ratio on the frame to help
        # with debugging and setting the correct eye aspect ratio
        # thresholds and frame counters
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 
    # show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
 
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
vs.stream.release()
vs.stop()
cv2.destroyAllWindows()

[INFO] starting video stream thread...
